# Setup

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from random import randint
import math
from datetime import datetime
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\DecisionTree'

In [3]:
for i in range(3):
    os.chdir("..")

In [4]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification'

In [5]:
main_data_dir = os.getcwd() + "\\Data set"
kmean_data_dir = main_data_dir + "\\kmean_dataset"
kmean_data_10cv_dir = kmean_data_dir + "\\10cv_64"
import glob

kmean_data_10cv_lst = [x for x in glob.glob(kmean_data_10cv_dir + '\\*') if 'surf' in x and 'index' in x and '100' in x]
print(kmean_data_10cv_lst)

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_test_100_4_3_True_Falsehisto_64_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf

In [6]:
kmean_train_paths = [x for x in kmean_data_10cv_lst if 'train' in x]
kmean_test_paths = [x for x in kmean_data_10cv_lst if 'test' in x]
print(kmean_train_paths)
print()
print(kmean_test_paths)
print(len(kmean_train_paths))
print(len(kmean_test_paths))

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\surf_train_100_4_3_True_Falsehisto_64_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_6

In [7]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """to_categorical _summary_

    Arguments:
        y -- The label of the data set with the shape of [None, 1]

    Keyword Arguments:
        num_classes -- The num_classes in the data set (default: {None})
        dtype -- the type of each element of the label after reshape (default: {'float32'})

    Returns:
        the label of the data set with the shape of [number of samples, number of classes].
    """    
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [8]:
class roc_curve_score:
    def __init__(self, y_true, y_pred, num_class):
        self.y_test = to_categorical(y_true)
        self.y_score = to_categorical(y_pred)
        print(np.unique(self.y_test), np.unique(self.y_score))
        self.fprs = {}
        self.tprs = {}
        self.thresh_holds = {}
        for x in range(num_class):
            self.fprs[x], self.tprs[x], self.thresh_holds[x] = roc_curve(self.y_test[:, x], self.y_score[:, x], drop_intermediate=False)
            self.fprs[x] = self.fprs[x].tolist()
            self.tprs[x] = self.tprs[x].tolist()
            self.thresh_holds[x] = self.thresh_holds[x].tolist()
        
        self.fpr_micro_avg, self.tpr_micro_avg, self.threshold_micro_avg, = roc_curve(self.y_test.ravel(), self.y_score.ravel())
        
        all_fpr = np.unique(np.concatenate([self.fprs[i] for i in range(num_class)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(num_class):
            mean_tpr += np.interp(all_fpr, self.fprs[i], self.tprs[i])
        mean_tpr /= num_class
        self.fpr_macro_avg = all_fpr.tolist()
        self.tpr_macro_avg = mean_tpr.tolist()
    
    def get_tpr(self, _class = None):
        if(_class):
            return self.tprs[_class]
        else:
            return self.tprs
    
    def get_fpr(self, _class = None):
        if(_class):
            return self.fprs[_class]
        else:
            return self.fprs

    def get_thresholds(self, _class = None):
        if(_class):
            return self.thresh_holds[_class]
        else:
            return self.thresh_holds

    def get_roc_dict(self):
        return {
            "tpr" : self.get_tpr(),
            "fpr" : self.get_fpr(),
            "thresholds" : self.get_thresholds(),
            "fpr_micro_avg" : self.fpr_micro_avg.tolist(),
            "tpr_micro_avg" : self.tpr_micro_avg.tolist(),
            "fpr_macro_avg" : self.fpr_macro_avg,
            "tpr_macro_avg" : self.tpr_macro_avg,
        }

In [9]:
class utils:
    def __init__(self, confusion_matrix, y_true, y_pred):
        """
            - confusion_matrix: 2x2 numpy array
            - y_true: array of label
            - y_pred: array of output value calculated by model
            - fold_count: number of folds
        """
        
        # Initilize all indicator
        self.TP = confusion_matrix[0][0] # true positive
        self.FN = confusion_matrix[1][0] # false negative
        self.FP = confusion_matrix[1][0] # false positive
        self.TN = confusion_matrix[1][1] # true negative
        self.precision = self.TP/(self.FN +  self.TP) # Precision Score - Positive Predictive Value
        self.recall = self.TP / (self.TP + self.FN)
        self.f1_score =  (2 * self.TP) / (2 * self.TP + self.FP + self.FN)
        self.sensitivity = self.TP / (self.TP + self.FN) # True Positive Rate
        self.specificity = self.TN / (self.TN + self.FP) # False Positive Rate
        self.negative_predictive_value = self.TN / (self.TN + self.FN) # Negative Predictive Value
        self.false_negative_rate = self.FN / (self.FN + self.TP) # False Negative Rate
        self.false_positive_rate = self.FP / (self.FP + self.TN) # False Positive Rate
        self.false_discovery_rate = self.FP / (self.FP + self.TP) # False Discovery Rate
        self.false_omission_rate = self.FN / (self.FN + self.TN) # False Ommision Rate
        self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
        self.negative_likelihood_ratio = self.false_negative_rate / self.specificity # Negative Likelihood Ratio
        self.prevalence_threshold = math.sqrt(self.false_positive_rate) / (math.sqrt(self.sensitivity) + 
                                                                      math.sqrt(self.false_positive_rate)) # Prevalance Threshold
        self.threat_score = self.TP / (self.TN + self.FN + self.FP) # Threat Score
        self.prevalence = (self.TP + self.FN)/(self.TP + self.FN + self.TN + self.FP) # Prevalance 
        #  Matthews correlation coefficient
        self.matthews_correlation_coefficient = (self.TP*self.TN - self.FN*self.FP) / ((self.TP + self.FP)
                                                                                       *(self.TP + self.FN)
                                                                                       *(self.TN + self.FP)
                                                                                       *(self.TN + self.FN))
        self.fowlkes_mallows_index = math.sqrt(self.sensitivity + self.precision) # Fowlkes–Mallows index
        self.informedness = self.sensitivity + self.specificity - 1 # informedness
        self.markedness = self.precision + self.negative_predictive_value - 1 # markedness
        self.diagnostic_odds_ratio = self.positive_likelihood_ratio / self.negative_likelihood_ratio # Diagnostic odds ratio
        self.accuracy = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        self.balanced_accuracy = (self.sensitivity + self.specificity) / 2
        self.roc_auc_macro = roc_auc_score(y_true, y_pred)
        self.roc_auc_micro = roc_auc_score(y_true, y_pred, average = 'micro')
        self.roc_auc_weighted = roc_auc_score(y_true, y_pred, average = 'weighted')
        self.cls_report = classification_report(y_true, y_pred)
        
        # Initilize the structure of output_dicts
        self.confusion_matrix = {
            "TP" : self.TP,
            "TN" : self.TN,
            "FN" : self.FN,
            "FP" : self.FP,
            "precision" : self.precision,
            "recall" : self.recall,
            "f1_score" : self.f1_score,
            "sensitivity" : self.sensitivity,
            "specificity" : self.specificity,
            "negative_predictive_value" : self.negative_predictive_value,
            "false_negative_rate" : self.false_negative_rate,
            "false_positive_rate" : self.false_positive_rate,
            "false_discovery_rate" : self.false_discovery_rate,
            "false_omission_rate" : self.false_omission_rate,
            "Positive_likelihood_ratio" : self.positive_likelihood_ratio,
            "Negative_likelihood_ratio" : self.negative_likelihood_ratio,
            "prevalence_threshold" : self.prevalence_threshold,
            "threat_score" : self.threat_score,
            "Prevalence" : self.prevalence,
            "Matthews_correlation_coefficient" : self.matthews_correlation_coefficient,
            "Fowlkes_Mallows_index" : self.fowlkes_mallows_index,
            "informedness" : self.informedness,
            "markedness" : self.markedness,
            "Diagnostic_odds_ratio" : self.diagnostic_odds_ratio,
            "accuracy" : self.accuracy,
            "balanced_accuracy" : self.balanced_accuracy
        }
        
        self.roc_auc_score = {
            "Macro": self.roc_auc_macro, 
            "Micro": self.roc_auc_micro,
            "Weight": self.roc_auc_weighted
        }
        
#         self.roc_curve = roc_curve_score(y_true=y_true, y_pred=y_pred, num_class=2).get_roc_dict()
        
        self.sub_dict = {
            "Confusion Matrix" : self.confusion_matrix,
            "ROC_AUC_SCORE" : self.roc_auc_score,
            "Classification Report" : classification_report(y_true.tolist(), y_pred.tolist(), 
                                                            labels = [0, 1], # 0 : Licit, 1 : Illicit
                                                            output_dict = True),
#             "ROC_DRAW" : self.roc_curve
        }
    def get_value(self):
        return self.sub_dict
        
# Test 
y_true = np.array([randint(0,1) for x in range(200)])
y_pred = np.array([randint(0,1) for x in range(200)])
confusion_matrix_test = confusion_matrix(y_true, y_pred)
base_utils = utils(confusion_matrix_test, y_true, y_pred)
base_utils.get_value()

{'Confusion Matrix': {'TP': 49,
  'TN': 43,
  'FN': 66,
  'FP': 66,
  'precision': 0.4260869565217391,
  'recall': 0.4260869565217391,
  'f1_score': 0.4260869565217391,
  'sensitivity': 0.4260869565217391,
  'specificity': 0.3944954128440367,
  'negative_predictive_value': 0.3944954128440367,
  'false_negative_rate': 0.5739130434782609,
  'false_positive_rate': 0.6055045871559633,
  'false_discovery_rate': 0.5739130434782609,
  'false_omission_rate': 0.6055045871559633,
  'Positive_likelihood_ratio': 0.7036890645586297,
  'Negative_likelihood_ratio': 1.4548028311425683,
  'prevalence_threshold': 0.5438146672079232,
  'threat_score': 0.28,
  'Prevalence': 0.5133928571428571,
  'Matthews_correlation_coefficient': -1.4313333118007513e-05,
  'Fowlkes_Mallows_index': 0.9231326627541018,
  'informedness': -0.1794176306342241,
  'markedness': -0.1794176306342241,
  'Diagnostic_odds_ratio': 0.48370064279155184,
  'accuracy': 0.4107142857142857,
  'balanced_accuracy': 0.41029118468288794},
 'RO

In [10]:
class Training:
    def __init__(self, fold_count, X, y):
        self.fold_count = fold_count
        self.param_grid = {
            "criterion" : ["gini", "entropy", "log_loss"],
            "splitter" : ["best", "random"],
            "max_features" : ["sqrt", "log2"],
            "class_weight" : ["balanced", None]
        }
        self.kf = KFold(n_splits=fold_count)
        self.history = {}
        self.X = X
        self.y = y
    
    def get_fold_value(self):
        return self.kf        
        
    def training(self, criterion,
                 splitter, 
                 max_features, 
                 class_weight, train_case):
    
        # output_dict initilize
        output_dict = {}
    
        # poiter track the index of fold
        fold_index = 0
    
        for train_index, test_index in zip(self.X, self.y):
            train_df = pd.read_csv(train_index)
            test_df = pd.read_csv(test_index)
            
            print("\tFold: {}".format(fold_index))
            print("\tTRAIN:", train_index, "\n\tTEST:", test_index)
        
            # folding data
            x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
            x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
    
            # Training
            print("\t\tTraining : {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
            model = DecisionTreeClassifier(criterion = criterion, 
                                              splitter = splitter, 
                                              max_features = max_features, 
                                              class_weight = class_weight)
            model.fit(x_train,y_train)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Testing
            print("\t\tValidation: {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
            y_pred = model.predict(x_test)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Evaluation
            cm = confusion_matrix(y_test, y_pred)
            current_utils = utils(cm, y_test, y_pred)
            output_dict["fold_{}".format(fold_index)] = current_utils.get_value()
            
            fold_index += 1
        print("\n")
        return output_dict 
    
    def train(self, path):
        count = 0
        for x in self.param_grid["criterion"]:
            for i in self.param_grid["splitter"]:
                for j in self.param_grid["max_features"]:
                    for k in self.param_grid["class_weight"]:
                        print("Traning Case: {}".format(count))
                        self.history["train_{}".format(count)] = {
                            "param" : {
                            "criterion" : x,
                            "splitter" : i,
                            "max_features" : j,
                            "class_weight" : k
                            },
                            "train_fold" : self.training(x, i, j, k, count)
                        }
                        count += 1
        with open(path.format(self.fold_count), 'wb') as f:
            pickle.dump(self.history, f)
        
        return self.history

# Training

In [11]:
training = Training(10, kmean_train_paths, kmean_test_paths)
approach_dir = os.getcwd() + "\\Experiment\\Approach"
svm_dir = approach_dir + "\\DecisionTree"
save_result_path = svm_dir + '\\rdf_km64_surf100_result_fold{0}_0.9.pkl'
training.train(path = save_result_path)

Traning Case: 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:12:22 --- End: 11/29/2022, 09:12:22
		Validation: 0 -- Start: 11/29/2022, 09:12:22---End: 11/29/2022, 09:12:22
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:12:22 --- End: 11/29/2022, 09:12:22
		Validation: 1 -- Start: 11/29/2022, 09:12:22---End: 11/29/2022, 09:12:22
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv 
	

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index2.csv
		Training : 2 -- Start: 11/29/2022, 09:12:23 --- End: 11/29/2022, 09:12:23
		Validation: 2 -- Start: 11/29/2022, 09:12:23---End: 11/29/2022, 09:12:23
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index3.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index3.csv
		Training : 3 -- Start: 11/29/2022, 09:12:23 --- End: 11/29/2022, 09:12:23
		Validation: 3 -- Start: 11/29/2022, 09:12:23---End: 11/29/2022, 09:12:23
	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index4.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index3.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index3.csv
		Training : 3 -- Start: 11/29/2022, 09:12:23 --- End: 11/29/2022, 09:12:23
		Validation: 3 -- Start: 11/29/2022, 09:12:23---End: 11/29/2022, 09:12:23
	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index4.csv
		Training : 4 -- Start: 11/29/2022, 09:12:23 --- End: 11/29/2022, 09:12:23
		Validation: 4 -- Start: 11/29/2022, 09:12:23---End: 11/29/2022, 09:12:23
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index5.csv 
	TEST: D:\Github\

...

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index4.csv
		Training : 4 -- Start: 11/29/2022, 09:12:27 --- End: 11/29/2022, 09:12:27
		Validation: 4 -- Start: 11/29/2022, 09:12:27---End: 11/29/2022, 09:12:27
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index5.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index5.csv
		Training : 5 -- Start: 11/29/2022, 09:12:27 --- End: 11/29/2022, 09:12:27
		Validation: 5 -- Start: 11/29/2022, 09:12:27---End: 11/29/2022, 09:12:27
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index6.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index5.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index5.csv
		Training : 5 -- Start: 11/29/2022, 09:12:27 --- End: 11/29/2022, 09:12:27
		Validation: 5 -- Start: 11/29/2022, 09:12:27---End: 11/29/2022, 09:12:27
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index6.csv
		Training : 6 -- Start: 11/29/2022, 09:12:27 --- End: 11/29/2022, 09:12:27
		Validation: 6 -- Start: 11/29/2022, 09:12:27---End: 11/29/2022, 09:12:27
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index7.csv 
	TEST: D:\Github\

{'train_0': {'param': {'criterion': 'gini',
   'splitter': 'best',
   'max_features': 'sqrt',
   'class_weight': 'balanced'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 3,
     'TN': 10,
     'FN': 2,
     'FP': 2,
     'precision': 0.6,
     'recall': 0.6,
     'f1_score': 0.6,
     'sensitivity': 0.6,
     'specificity': 0.8333333333333334,
     'negative_predictive_value': 0.8333333333333334,
     'false_negative_rate': 0.4,
     'false_positive_rate': 0.16666666666666666,
     'false_discovery_rate': 0.4,
     'false_omission_rate': 0.16666666666666666,
     'Positive_likelihood_ratio': 3.6,
     'Negative_likelihood_ratio': 0.48,
     'prevalence_threshold': 0.3451409985003952,
     'threat_score': 0.21428571428571427,
     'Prevalence': 0.29411764705882354,
     'Matthews_correlation_coefficient': 0.007222222222222222,
     'Fowlkes_Mallows_index': 1.0954451150103321,
     'informedness': 0.43333333333333335,
     'markedness': 0.43333333333333335,
     'Diagnostic_od

# Result Analyzing

In [12]:
file = open(save_result_path.format(10), 'rb')
result = pickle.load(file)
result

{'train_0': {'param': {'criterion': 'gini',
   'splitter': 'best',
   'max_features': 'sqrt',
   'class_weight': 'balanced'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 3,
     'TN': 10,
     'FN': 2,
     'FP': 2,
     'precision': 0.6,
     'recall': 0.6,
     'f1_score': 0.6,
     'sensitivity': 0.6,
     'specificity': 0.8333333333333334,
     'negative_predictive_value': 0.8333333333333334,
     'false_negative_rate': 0.4,
     'false_positive_rate': 0.16666666666666666,
     'false_discovery_rate': 0.4,
     'false_omission_rate': 0.16666666666666666,
     'Positive_likelihood_ratio': 3.6,
     'Negative_likelihood_ratio': 0.48,
     'prevalence_threshold': 0.3451409985003952,
     'threat_score': 0.21428571428571427,
     'Prevalence': 0.29411764705882354,
     'Matthews_correlation_coefficient': 0.007222222222222222,
     'Fowlkes_Mallows_index': 1.0954451150103321,
     'informedness': 0.43333333333333335,
     'markedness': 0.43333333333333335,
     'Diagnostic_od

In [13]:
print("Number of Training: {}".format(len(list(result.keys()))))

Number of Training: 24


In [14]:
training_case_lst = list(result.keys())
print(training_case_lst[0])

train_0


In [15]:
train_sample = result[training_case_lst[0]]
print(train_sample['param'])
print("Number of Fold: {}".format(len(train_sample['train_fold'])))

{'criterion': 'gini', 'splitter': 'best', 'max_features': 'sqrt', 'class_weight': 'balanced'}
Number of Fold: 10


In [16]:
fold_sample = train_sample['train_fold']['fold_0']
print(fold_sample['Classification Report'])

{'0': {'precision': 0.6, 'recall': 0.5, 'f1-score': 0.5454545454545454, 'support': 6}, '1': {'precision': 0.7692307692307693, 'recall': 0.8333333333333334, 'f1-score': 0.8, 'support': 12}, 'accuracy': 0.7222222222222222, 'macro avg': {'precision': 0.6846153846153846, 'recall': 0.6666666666666667, 'f1-score': 0.6727272727272727, 'support': 18}, 'weighted avg': {'precision': 0.7128205128205128, 'recall': 0.7222222222222222, 'f1-score': 0.7151515151515153, 'support': 18}}


In [17]:
result_analyze_dict = {
    'Train' : [],
    'MACC' : [],
    'MAP' : [],
    'MASens' : [],
    'MASpec' : [],
    'MAF1' : [],
    'AUC' : []
}

for result_key in result:
    result_analyze_dict['Train'].append(result_key)
    train_base = result[result_key]
    fold_base = train_base['train_fold']
    acc_lst, pre_lst, re_lst, spec_lst, f1_lst, auc_lst = [], [], [], [], [], []
    for fold_key in fold_base:
        current_fold = fold_base[fold_key]
        current_fold_cls = current_fold['Classification Report']
        acc_lst.append(current_fold_cls['accuracy'])
        pre_lst.append(current_fold_cls['macro avg']['precision'])
        re_lst.append(current_fold_cls['macro avg']['recall'])
        spec_lst.append(current_fold['Confusion Matrix']['specificity'])
        auc_lst.append(current_fold['ROC_AUC_SCORE']['Macro'])
        f1_lst.append(current_fold_cls['macro avg']['f1-score'])
    result_analyze_dict['MACC'].append(sum(acc_lst)/len(acc_lst))
    result_analyze_dict['MAP'].append(sum(pre_lst)/len(pre_lst))
    result_analyze_dict['MASens'].append(sum(re_lst)/len(re_lst))
    result_analyze_dict['MASpec'].append(sum(spec_lst)/len(spec_lst))
    result_analyze_dict['MAF1'].append(sum(f1_lst)/len(f1_lst))
    result_analyze_dict['AUC'].append(sum(auc_lst)/len(auc_lst))

result_df = pd.DataFrame(result_analyze_dict)
result_df.head()

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
0,train_0,0.783333,0.757887,0.750000,0.850000,0.750478,0.750000
1,train_1,0.816667,0.814910,0.795833,0.858333,0.793898,0.795833
2,train_2,0.838034,0.820442,0.807576,0.881818,0.804710,0.807576
3,train_3,0.842308,0.838028,0.816288,0.899242,0.811286,0.816288
4,train_4,0.782479,0.756722,0.732576,0.865152,0.732095,0.732576


In [18]:
result_max = result_df.max()
result_max

Train      train_9
MACC      0.842308
MAP       0.838028
MASens    0.816288
MASpec    0.924242
MAF1      0.811286
AUC       0.816288
dtype: object

In [19]:
id_result_max = result_df.iloc[:, 1:].idxmax()
id_result_max

MACC       3
MAP        3
MASens     3
MASpec    22
MAF1       3
AUC        3
dtype: int64

In [20]:
id_max = 'train_{}'.format(id_result_max.mode()[0])
print(id_max)

train_3


In [21]:
result_df.loc[result_df['Train'] == id_max]

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
3,train_3,0.842308,0.838028,0.816288,0.899242,0.811286,0.816288


In [22]:
train_max = result[id_max]
train_max['param']

{'criterion': 'gini',
 'splitter': 'best',
 'max_features': 'log2',
 'class_weight': None}

In [23]:
train_max_fold = train_max['train_fold']
train_max_analyze = {
    'Fold' : [],
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for fold_key in train_max_fold:
    train_max_analyze['Fold'].append(fold_key)
    current_fold = train_max_fold[fold_key]
    current_fold_cls = current_fold['Classification Report']
    train_max_analyze['ACC'].append(current_fold_cls['accuracy'])
    train_max_analyze['AP'].append(current_fold_cls['macro avg']['precision'])
    train_max_analyze['ASens'].append(current_fold_cls['macro avg']['recall'])
    train_max_analyze['ASpec'].append(current_fold['Confusion Matrix']['specificity'])
    train_max_analyze['AF1'].append(current_fold_cls['macro avg']['f1-score'])
    train_max_analyze['AUC'].append(current_fold['ROC_AUC_SCORE']['Macro'])

train_max_analyze_df = pd.DataFrame(train_max_analyze)
train_max_analyze_df

,Fold,ACC,AP,ASens,ASpec,AF1,AUC
0,fold_0,0.888889,0.875000,0.875000,0.916667,0.875000,0.875000
1,fold_1,0.888889,0.928571,0.833333,1.000000,0.861538,0.833333
2,fold_2,0.777778,0.750000,0.750000,0.833333,0.750000,0.750000
3,fold_3,0.722222,0.722222,0.750000,0.666667,0.714286,0.750000
4,fold_4,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
5,fold_5,0.777778,0.875000,0.666667,1.000000,0.678571,0.666667
6,fold_6,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
7,fold_7,0.888889,0.875000,0.875000,0.916667,0.875000,0.875000
8,fold_8,0.888889,0.875000,0.875000,0.916667,0.875000,0.875000
9,fold_9,0.923077,0.833333,0.954545,0.909091,0.876190,0.954545


# N_times_K_folds

In [24]:
# Setup

train_max_param = train_max['param']
n_times = 100

nt_kf_output_dict = {
    'Train' : [], # <index>_<fold>
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for n in range(n_times):
    print("Training {}".format(n))
    fold_index = 0
    for train_index, test_index in zip(kmean_train_paths, kmean_test_paths):
        train_df = pd.read_csv(train_index)
        test_df = pd.read_csv(test_index)      
    
        print("\tFold: {}".format(fold_index))
        print("\tTRAIN:", train_index, "\n\tTEST:", test_index)

        # folding data
        x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
        x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
        
        # Training
        print("\t\tTraining : {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
        model = DecisionTreeClassifier(criterion = train_max_param['criterion'], 
                                      splitter = train_max_param['splitter'], 
                                      max_features = train_max_param['max_features'], 
                                      class_weight = train_max_param['class_weight'])
        model.fit(x_train, y_train)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))

        # Testing
        print("\t\tValidation: {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
        y_pred = model.predict(x_test)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
        # Evaluation
        cm = confusion_matrix(y_test, y_pred)
        base_result = utils(cm, y_test, y_pred).get_value()
        
        # Result Storing
        nt_kf_output_dict['Train'].append('{0}_{1}'.format(n, fold_index))
        cls = base_result['Classification Report']
        nt_kf_output_dict['ACC'].append(cls['accuracy'])
        nt_kf_output_dict['AP'].append(cls['macro avg']['precision'])
        nt_kf_output_dict['ASens'].append(cls['macro avg']['recall'])
        nt_kf_output_dict['ASpec'].append(base_result['Confusion Matrix']['specificity'])
        nt_kf_output_dict['AF1'].append(cls['macro avg']['f1-score'])
        nt_kf_output_dict['AUC'].append(base_result['ROC_AUC_SCORE']['Macro'])
        
        # Fold index increment
        fold_index += 1

Training 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:12:29 --- End: 11/29/2022, 09:12:29
		Validation: 0 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:12:29 --- End: 11/29/2022, 09:12:29
		Validation: 1 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv 
	TEST:

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index0.csv
		Training : 0 -- Start: 11/29/2022, 09:12:29 --- End: 11/29/2022, 09:12:29
		Validation: 0 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:12:29 --- End: 11/29/2022, 09:12:29
		Validation: 1 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/29/2022, 09:12:29
		Validation: 2 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index3.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index3.csv
		Training : 3 -- Start: 11/29/2022, 09:12:29 --- End: 11/29/2022, 09:12:29
		Validation: 3 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 4
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index4.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index4.csv
		Training : 4 -- Start: 11/29/2022, 09:12:29 --- End: 11/29/2022, 09:12:29
		Validation: 4 -- Start: 11/29/2022, 09:12:29---End: 11/29/2022, 09:12:29
	Fold: 5
	TRAIN: D:\Github\Stomach-Status-Classification\

...

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:12:48 --- End: 11/29/2022, 09:12:48
		Validation: 1 -- Start: 11/29/2022, 09:12:48---End: 11/29/2022, 09:12:48
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index2.csv
		Training : 2 -- Start: 11/29/2022, 09:12:48 --- End: 11/29/2022, 09:12:48
		Validation: 2 -- Start: 11/29/2022, 09:12:48---End: 11/29/2022, 09:12:48
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index3.csv 
	TEST: D:\Github\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/29/2022, 09:12:48
		Validation: 0 -- Start: 11/29/2022, 09:12:48---End: 11/29/2022, 09:12:48
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index1.csv
		Training : 1 -- Start: 11/29/2022, 09:12:48 --- End: 11/29/2022, 09:12:48
		Validation: 1 -- Start: 11/29/2022, 09:12:48---End: 11/29/2022, 09:12:48
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_train_100_4_3_True_Falsehisto_64_cv_index2.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\surf_test_100_4_3_True_Falsehisto_64_cv_index2.csv
		Training : 2 -- Start: 11/29/2022, 09:12:48 --- End: 11/29/2022, 09:12:48
		Validation: 2 -- Start: 11/29/2022, 09:12:48---End: 11/29/2022, 09:12:48
	Fold: 3
	TRAIN: D:\Github\Stomach-Status-Classification\

C:\Users\kohi2\AppData\Local\Temp\ipykernel_11556\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


In [25]:
nt_kf_output_df = pd.DataFrame(nt_kf_output_dict)
nt_kf_output_df.head(10)

,Train,ACC,AP,ASens,ASpec,AF1,AUC
0,0_0,0.666667,0.625000,0.625000,0.750000,0.625000,0.625000
1,0_1,0.777778,0.767857,0.708333,0.916667,0.723077,0.708333
2,0_2,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
3,0_3,0.722222,0.722222,0.750000,0.666667,0.714286,0.750000
4,0_4,0.833333,0.823077,0.791667,0.916667,0.803636,0.791667
5,0_5,0.722222,0.684615,0.666667,0.833333,0.672727,0.666667
6,0_6,0.944444,0.961538,0.916667,1.000000,0.934545,0.916667
7,0_7,0.888889,0.875000,0.875000,0.916667,0.875000,0.875000
8,0_8,0.777778,0.767857,0.708333,0.916667,0.723077,0.708333
9,0_9,0.923077,0.833333,0.954545,0.909091,0.876190,0.954545


In [26]:
nt_kf_output_df.mean()

Train         inf
ACC      0.787620
AP       0.759278
ASens    0.747883
ASpec    0.849765
AF1      0.742490
AUC      0.747883
dtype: float64

In [27]:
nt_kf_output_df.std(numeric_only=True)

ACC      0.092725
AP       0.120783
ASens    0.118187
ASpec    0.104349
AF1      0.118444
AUC      0.118187
dtype: float64

In [28]:
acc_values = nt_kf_output_df['ACC'].values.tolist()
ap_values = nt_kf_output_df['AP'].values.tolist()
asens_values = nt_kf_output_df['ASens'].values.tolist()
aspec_values = nt_kf_output_df['ASpec'].values.tolist()
af1_values = nt_kf_output_df['AF1'].values.tolist()
auc_values = nt_kf_output_df['AUC'].values.tolist()
print(len(acc_values), len(ap_values), len(asens_values), len(aspec_values), len(af1_values), len(auc_values))

1000 1000 1000 1000 1000 1000


In [29]:
import scipy.stats as st

def CI(data, confidence_level, dis_type = 't'):
    if dis_type == 't':
        return st.t.interval(confidence=confidence_level, df=len(data)-1, 
                             loc=np.mean(data), scale=st.sem(data))
    elif dis_type == 'g':
        return st.norm.interval(confidence=confidence_level, loc=np.mean(data), scale=st.sem(data))

def CI_calculator(confidence_level, dis_type = 't'):
    for x in nt_kf_output_df.columns[1:]:
        base = nt_kf_output_df[x].values.tolist()
        print("CI of {}".format(x), CI(base, confidence_level, dis_type))

In [30]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 't')

CI of ACC (0.781865637616856, 0.7933736786224603)
CI of AP (0.7517828870204236, 0.7667732131702353)
CI of ASens (0.7405485236054014, 0.7552166279097502)
CI of ASpec (0.8432898205381012, 0.8562404824922019)
CI of AF1 (0.7351396499314263, 0.7498396053699746)
CI of AUC (0.7405485236054014, 0.7552166279097502)


In [31]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7818726089056629, 0.7933667073336534)
CI of AP (0.7517919677921074, 0.7667641323985515)
CI of ASens (0.7405574091829993, 0.7552077423321523)
CI of ASpec (0.8432976657312734, 0.8562326372990297)
CI of AF1 (0.7351485548036597, 0.7498307004977413)
CI of AUC (0.7405574091829993, 0.7552077423321523)


In [32]:
cl = 0.99

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.780066755162709, 0.7951725610766074)
CI of AP (0.7494396702488022, 0.7691164299418567)
CI of ASens (0.7382556750262346, 0.757509476488917)
CI of ASpec (0.8412654344112637, 0.8582648686190394)
CI of AF1 (0.7328418225338466, 0.7521374327675544)
CI of AUC (0.7382556750262346, 0.757509476488917)
